<a href="https://colab.research.google.com/github/TrevorSeestedt/CSCE580-Fall2024-TrevorSeestedt-Repo/blob/main/TrevorSeestedt_Quiz2_Response.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section Using search for a practical problem
Problem: Missionaries and cannibals problem. Three missionaries and three cannibals are on one side of a river, along iwth a boat that can hold one or two people. Find a way to get everyone to the other side, without ever leaving a group of missionaries in one place outnumbered by the cannibals in the same place.

Notes:
- States: a state consists of an order equence of three numbers representing the number of missionaries, cannibals, and boats on the bank of the river from which they started. Thus, start state: (3,3,1).
- Operators: from each state the possible operators are to take either one missionary, one cannibal, two missionarioes, two cannibals, or one across in the boat, thus there are at most five operators.
- Goal test: reached state of (0,0,0).
- Path cost: number of crossings.

In [16]:
# Part a
# Use any uninformed search method to solve this problem - Using Breadth Depth Search
import time

start_state = (3,3,1) # (missionaries_left, cannibals left, boat left)
goal_state = (0,0,0)

def is_valid(state):
  missionaries_left, cannibals_left, boat_left = state
  missionaries_right = 3 - missionaries_left
  cannibals_right = 3 - cannibals_left

  # checking for no negatives
  if (missionaries_left < 0 or cannibals_left < 0 or missionaries_right < 0 or cannibals_right < 0):
    return False

  # if there are more cannibals than missionaries on left then fails
  if (missionaries_left > 0 and missionaries_left < cannibals_left):
    return False;

 # if there are more cannibals than missionaries on the right then fails.
  if (missionaries_right > 0 and missionaries_right < cannibals_right):
    return False;

  return True;

def get_successors(state):
  successors = []
  missionaries_left, cannibals_left, boat_left = state # tuple to represent the current state of the problem

  # determine the direction based on the boat's position
  # boat on left side
  if boat_left:
      missionaries_available = missionaries_left
      cannibals_available = cannibals_left
      m_move_range = range(0, min(2, missionaries_available) + 1)
      c_move_range = range(0, min(2, cannibals_available) + 1)
      boat_to = 0 # boat looks to move to the right bank
  # boat on right side
  else:
      missionaries_available = 3 - missionaries_left
      cannibals_available = 3 - cannibals_left
      m_move_range = range(0, min(2, missionaries_available) + 1)
      c_move_range = range(0, min(2, cannibals_available) + 1)
      boat_to = 1

  # combinations of missionaries and cannibals
  for missionaries in m_move_range:
    for cannibals in c_move_range:
      total = missionaries + cannibals
      #boat can only carry 1 or 2 people
      if total >= 1 and total <= 2:
        if missionaries == 0 and cannibals ==0:
          continue
        if boat_left:
          new_state = (missionaries_left - missionaries, cannibals_left - cannibals, boat_to)
        else:
          new_state = (missionaries_left + missionaries, cannibals_left + cannibals, boat_to)
        if is_valid(new_state):
          successors.append(new_state)
  return successors

def bfs(start, goal):
  queue = []
  visited = []
  parent = {}

  queue.append(start)
  visited.append(start)

  while queue:
    state = queue.pop(0)

    if state == goal:
      path = []
      while state != start:
        path.append(state)
        state = parent[state]
      path.append(start)
      path.reverse()
      return path

    for successor in get_successors(state):
      if successor not in visited:
        queue.append(successor)
        visited.append(successor)
        parent[successor] = state
  return None

start_time = time.time()
sol = bfs(start_state, goal_state)
end_time = time.time()
elapsed_time = end_time - start_time
print(sol)
print(f"Time for BFS: {elapsed_time} seconds")

[(3, 3, 1), (3, 1, 0), (3, 2, 1), (3, 0, 0), (3, 1, 1), (1, 1, 0), (2, 2, 1), (0, 2, 0), (0, 3, 1), (0, 1, 0), (0, 2, 1), (0, 0, 0)]
Time for BFS: 0.000148773193359375 seconds


In [17]:
# Part b
# Use any informed search method and solve the problem - Using A Star search algorithm
import time

start_state = (3,3,1)  # (missionaries_left, cannibals_left, boat_left)
goal_state = (0,0,0)

def is_valid(state):
  missionaries_left, cannibals_left, boat_left = state
  missionaries_right = 3 - missionaries_left
  cannibals_right = 3 - cannibals_left

  # checking for no negatives
  if (missionaries_left < 0 or cannibals_left < 0 or missionaries_right < 0 or cannibals_right < 0):
    return False

  # if there are more cannibals than missionaries on left then fails
  if (missionaries_left > 0 and missionaries_left < cannibals_left):
    return False

 # if there are more cannibals than missionaries on the right then fails
  if (missionaries_right > 0 and missionaries_right < cannibals_right):
    return False

  return True

def get_successors(state):
  successors = []
  missionaries_left, cannibals_left, boat_left = state  # tuple to represent the current state of the problem

  # boats on left side
  if boat_left:
    missionaries_available = missionaries_left
    cannibals_available = cannibals_left
    # boat goes to right
    boat_to = 0

  # boats on right side
  else:
    missionaries_available = 3 - missionaries_left
    cannibals_available = 3 - cannibals_left
    # boat goes to left
    boat_to = 1

  # combine the missionaries and cannibals
  for missionaries in range(0, min(2, missionaries_available) + 1):
      for cannibals in range(0, min(2, cannibals_available) + 1):
          total = missionaries + cannibals
          if total >= 1 and total <= 2:
              if missionaries == 0 and cannibals == 0:
                  continue
              if boat_left:
                  new_state = (missionaries_left - missionaries, cannibals_left - cannibals, boat_to)
              else:
                  new_state = (missionaries_left + missionaries, cannibals_left + cannibals, boat_to)
              if is_valid(new_state):
                  successors.append(new_state)

  return successors

def heuristic(state):
  # tuple created for each state
  missionaries_left, cannibals_left, boat_left = state
  return missionaries_left + cannibals_left

def a_star(start, goal):
  parent = {}
  g_cost = {}
  f_cost = {}

  open_list = []
  closed_list = set()

  open_list.append(start)
  g_cost[start] = 0
  f_cost[start] = heuristic(start)

  while open_list:
    current = min(open_list, key=lambda x: f_cost.get(x, float('inf')))
    if current == goal:
      path = []
      while current != start:
        path.append(current)
        current = parent[current]
      path.append(start)
      path.reverse()
      return path

    open_list.remove(current)
    closed_list.add(current)

    for successor in get_successors(current):
      if successor in closed_list:
        continue

      # tentative g cost, each move costs 1
      tg_cost = g_cost[current] + 1

      if successor not in open_list or tg_cost < g_cost.get(successor, float('inf')):
        parent[successor] = current
        g_cost[successor] = tg_cost
        f_cost[successor] = tg_cost + heuristic(successor)

        if successor not in open_list:
          open_list.append(successor)

  return None  # no solution

start_time = time.time()
sol = a_star(start_state, goal_state)
end_time = time.time()
elapsed_time = end_time - start_time
print(sol)
print(f"Time for A-star: {elapsed_time} seconds")


[(3, 3, 1), (3, 1, 0), (3, 2, 1), (3, 0, 0), (3, 1, 1), (1, 1, 0), (2, 2, 1), (0, 2, 0), (0, 3, 1), (0, 1, 0), (0, 2, 1), (0, 0, 0)]
Time for A-star: 0.0002589225769042969 seconds
